In [ ]:
!rm -rf fast_slow_rate/ oracle.tsv

In [1]:
!git clone https://github.com/chrisliu298/fast_slow_rate.git
!cp fast_slow_rate/oracle25000.tsv .

Cloning into 'fast_slow_rate'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 107 (delta 1), reused 2 (delta 0), pack-reused 101
Receiving objects: 100% (107/107), 49.03 MiB | 13.70 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Checking out files: 100% (13/13), done.


In [2]:
# Models
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Dataset
import tensorflow_datasets as tfds

# Others
from statistics import mean
import json
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import pandas as pd


def plot(history, string):
    """
    Plot training acc/loss and validation acc/loss
    """
    plt.plot(history.history[string])
    plt.plot(history.history["val_" + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend(["train_" + string, "valid_" + string])
    plt.show()


class GridSearch:
    def __init__(
        self,
        train_size,
        valid_size,
        test_size,
        lstm_search_space,
        dense_search_space,
        dropout_search_space,
        batch_size,
        learning_rate,
    ):
        # Parameters
        self.train_size = train_size
        self.valid_size = valid_size
        self.test_size = test_size
        self.lstm_search_space = lstm_search_space
        self.dense_search_space = dense_search_space
        self.dropout_search_space = dropout_search_space
        self.vocab_size = 20000
        self.embedding_dim = 50
        self.max_len = 512
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = 50
        self.patience = 10
        self.seed = 42
        self.trunc_type = "post"
        self.padding = "pre"
        self.oov_token = "<OOV>"
        self.callbacks = [
            EarlyStopping(
                monitor="val_accuracy",
                patience=self.patience,
                restore_best_weights=True,
            )
        ]
        # Load the dataset
        imdb = tfds.load("imdb_reviews", as_supervised=True, shuffle_files=True)
        oracle_train = pd.read_csv("oracle25000.tsv", delimiter="\t")
        self.oracle_text = oracle_train["reviews"]
        self.oracle_labels = oracle_train["oracle_labels"]
        self.train_dataset = imdb["train"]
        self.test_dataset = imdb["test"]

    def prepare_data(self):
        train_text = self.oracle_text
        train_labels = self.oracle_labels
        test_text = []
        test_labels = []

        # Put sentences and labels in lists
        for s, l in self.test_dataset:
            test_text.append(str(s.numpy()))
            test_labels.append(l.numpy())

        # Convert them into numpy arrays
        train_text, train_labels, test_text, test_labels = (
            np.array(train_text),
            np.array(train_labels),
            np.array(test_text),
            np.array(test_labels),
        )

        # Shuffle the train/test set
        train_rand = np.arange(len(train_text))
        np.random.shuffle(train_rand)
        train_text = train_text[train_rand]
        train_labels = train_labels[train_rand]
        
        test_rand = np.arange(len(test_text))
        np.random.shuffle(test_rand)
        test_text = test_text[test_rand]
        test_labels = test_labels[test_rand]

        # Take the subset of the data
        train_reviews, valid_reviews = (
            train_text[: self.train_size],
            train_text[-self.valid_size :],
        )
        train_sentiments, valid_sentiments = (
            np.array(train_labels[: self.train_size]),
            np.array(train_labels[-self.valid_size :]),
        )
        test_reviews = test_text[: self.test_size]
        test_sentiments = np.array(test_labels[: self.test_size])

        tokenizer = Tokenizer(num_words=self.vocab_size, oov_token=self.oov_token)
        tokenizer.fit_on_texts(train_reviews)

        train_seq = tokenizer.texts_to_sequences(train_reviews)
        train_padded = pad_sequences(
            train_seq,
            maxlen=self.max_len,
            truncating=self.trunc_type,
            padding=self.padding,
        )
        valid_seq = tokenizer.texts_to_sequences(valid_reviews)
        valid_padded = pad_sequences(
            valid_seq,
            maxlen=self.max_len,
            truncating=self.trunc_type,
            padding=self.padding,
        )
        test_seq = tokenizer.texts_to_sequences(test_reviews)
        test_padded = pad_sequences(
            test_seq,
            maxlen=self.max_len,
            truncating=self.trunc_type,
            padding=self.padding,
        )

        print("Training sentences count:", len(train_padded))
        print("Training labels count:", len(train_sentiments))
        print("Validation sentences count:", len(valid_padded))
        print("Validation labels count:", len(valid_sentiments))
        print("Test sentences count:", len(test_padded))
        print("Testing labels count:", len(test_sentiments))
        print()
        return [
            (train_padded, train_sentiments),
            (valid_padded, valid_sentiments),
            (test_padded, test_sentiments),
        ]

    def build_model(self, lstm_hidden_size, dense_hidden_size, dropout_rate):
        model = Sequential()
        model.add(
            Embedding(self.vocab_size, self.embedding_dim, input_length=self.max_len)
        )
        model.add(Bidirectional(LSTM(lstm_hidden_size, return_sequences=True)))
        model.add(GlobalMaxPool1D())
        model.add(Dense(dense_hidden_size, activation="relu"))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation="sigmoid"))
        model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(learning_rate=self.learning_rate),
            metrics=["accuracy"],
        )
        return model

    def train(self, data):
        (
            (train_padded, train_sentiments),
            (valid_padded, valid_sentiments),
            (test_padded, test_sentiments),
        ) = data
        histories = []
        counter = 0
        for lstm_size in self.lstm_search_space:
            for dense_size in self.dense_search_space:
                for dropout_rate in self.dropout_search_space:
                    model = self.build_model(lstm_size, dense_size, dropout_rate)
                    counter += 1
                    print(f"========== Trial {counter} Summary ==========")
                    print(f"   lstm_search_space = {lstm_size}")
                    print(f"  dense_search_space = {dense_size}")
                    print(f"dropout_search_space = {dropout_rate}\n")
                    history = model.fit(
                        train_padded,
                        train_sentiments,
                        validation_data=(valid_padded, valid_sentiments),
                        batch_size=self.batch_size,
                        epochs=self.epochs,
                        shuffle=True,
                        callbacks=self.callbacks,
                        verbose=0
                    )
                    log = {
                        "lstm_hidden_size": lstm_size,
                        "dense_hidden_size": dense_size,
                        "dropout_rate": dropout_rate,
                        "train_acc_early_stop": history.history["accuracy"][-11],
                        "train_acc_final": history.history["accuracy"][-1],
                        "valid_acc_early_stop": history.history["val_accuracy"][-11],
                        "valid_acc_final": history.history["val_accuracy"][-1],
                        "max_valid_acc": max(history.history["val_accuracy"]),
                    }
                    histories.append(log)
                    print()
                    print(f"========== Trial {counter} Results ==========")
                    print(json.dumps(log, indent=4))
                    print()
        return histories

    def get_best_model(self, history):
        return max(history, key=lambda x: x["max_valid_acc"])

    def train_best_model(self, best_params, num_trials=100):
        best_lstm_size = best_params["lstm_hidden_size"]
        best_dense_size = best_params["dense_hidden_size"]
        best_dropout_rate = best_params["dropout_rate"]
        (train_padded, train_sentiments), (valid_padded, valid_sentiments), (test_padded, test_sentiments) = self.prepare_data()
        train_acc = []
        final_train_acc = []
        valid_acc = []
        final_valid_acc = []
        test_acc = []
        for t in range(num_trials):
            print(f"========== Tiral {t + 1} ==========")
            model = self.build_model(best_lstm_size, best_dense_size, best_dropout_rate)
            history = model.fit(
                train_padded,
                train_sentiments,
                validation_data=(valid_padded, valid_sentiments),
                batch_size=self.batch_size,
                epochs=self.epochs,
                shuffle=True,
                callbacks=self.callbacks,
                verbose=0
            )
            loss, acc = model.evaluate(test_padded, test_sentiments, batch_size=self.batch_size)
            train_acc.append(history.history["accuracy"][-11])
            final_train_acc.append(history.history["accuracy"][-1])
            valid_acc.append(history.history["val_accuracy"][-11])
            final_valid_acc.append(history.history["val_accuracy"][-1])
            test_acc.append(acc)
            print()
        return (train_acc, final_train_acc, valid_acc, final_valid_acc, test_acc)


In [ ]:
start = timer()
search = GridSearch(
    train_size=100,
    valid_size=2000,
    test_size=12500,
    lstm_search_space=[16],
    dense_search_space=[128],
    dropout_search_space=[0.1],
    batch_size=32,
    learning_rate=3e-4,
)

data = search.prepare_data()
history = search.train(data)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDE6NRE/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDE6NRE/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDE6NRE/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Training sentences count: 11000
Training labels count: 11000
Validation sentences count: 2000
Validation labels count: 2000
Test sentences count: 12500
Testing labels count: 12500

========== Trial 1 Summary ==========
   lstm_search_space = 16
  dense_search_space = 128
dropout_search_space = 0.4


========== Trial 1 Results ==========
{
    "lstm_hidden_size": 16,
    "dense_hidden_size": 128,
    "dropout_rate": 0.4,
    "train_acc_early_stop": 0.9293636083602905,
    "train_acc_final": 0.9987272620201111,
    "valid_acc_early_stop": 0.8980000019073486,
    "valid_acc_final": 0.8824999928474426,
    "max_valid_acc": 0.8980000019073486
}



In [ ]:
best_params = search.get_best_model(history)
print(json.dumps(best_params, indent=4))

{
    "lstm_hidden_size": 16,
    "dense_hidden_size": 128,
    "dropout_rate": 0.4,
    "train_acc_early_stop": 0.9293636083602905,
    "train_acc_final": 0.9987272620201111,
    "valid_acc_early_stop": 0.8980000019073486,
    "valid_acc_final": 0.8824999928474426,
    "max_valid_acc": 0.8980000019073486
}


In [ ]:
(train_acc, final_train_acc, valid_acc, final_valid_acc, test_acc) = search.train_best_model(best_params)

Training sentences count: 11000
Training labels count: 11000
Validation sentences count: 2000
Validation labels count: 2000
Test sentences count: 12500
Testing labels count: 12500

========== Tiral 1 ==========
391/391 [==============================] - 7s 17ms/step - loss: 0.9320 - accuracy: 0.8426

========== Tiral 2 ==========
391/391 [==============================] - 7s 17ms/step - loss: 0.9275 - accuracy: 0.8386

========== Tiral 3 ==========
391/391 [==============================] - 7s 17ms/step - loss: 1.3882 - accuracy: 0.8399

========== Tiral 4 ==========
391/391 [==============================] - 7s 17ms/step - loss: 1.1099 - accuracy: 0.8448

========== Tiral 5 ==========
391/391 [==============================] - 7s 17ms/step - loss: 1.2279 - accuracy: 0.8402

========== Tiral 6 ==========
391/391 [==============================] - 7s 17ms/step - loss: 0.9688 - accuracy: 0.8359

========== Tiral 7 ==========
391/391 [==============================] - 7s 17ms/step - loss:

In [ ]:
from statistics import mean

In [ ]:
print(search.train_size)
print(round(mean(train_acc), 4))
print(round(mean(final_train_acc), 4))
print(round(mean(valid_acc), 4))
print(round(mean(final_valid_acc), 4))
print(round(mean(test_acc), 4))

11000
0.9684
0.999
0.8773
0.868
0.8366


In [ ]:
end = timer()
print((end - start) / 3600)

7.349984844821666
